In [1]:
import pandas as pd

In [2]:
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf
symbol='TATACOFFEE.NS'
start = dt.datetime(2013,6,1)
end = dt.datetime(2022,2,11)
stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [4]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (2144, 4)


/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/utils/valid

In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [6]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

1715
X_train length: (1715, 4)
X_test length: (429, 4)
y_train length: (1715, 4)
y_test length: (429, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [9]:
from statsmodels.tsa.statespace.varmax import VARMAX

def varma_model(dataset, listt):
    print(listt)
    dataset_two = dataset[listt]
    test_obs = 28
    train = dataset_two[:-test_obs]
    test = dataset_two[-test_obs:]

    # Finding optimal order (p, q) for VARMA
    best_aic = float('inf')
    best_order = (0, 0)

    for p in range(1, 4):  # Adjusting the range based on our data
        for q in range(1, 4):  # Adjusting the range based on our data
            try:
                model = VARMAX(train, order=(p, q))
                results = model.fit(maxiter=1000, disp=False)
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_order = (p, q)
            except:
                continue

    print('Best Order (p, q):', best_order)
    model_varma = VARMAX(train, order=best_order)
    results_varma = model_varma.fit(maxiter=1000, disp=False)

    # Forecasting
    pred_varma = results_varma.get_forecast(steps=28).predicted_mean
    pred_varma.to_csv("varma_forecasted_{}.csv".format(test_obs))

    # Evaluating performance
    from sklearn.metrics import mean_squared_error
    rmse_varma = round(mean_squared_error(test, pred_varma, squared=False), 2)
    from sklearn.metrics import mean_absolute_percentage_error
    mape_varma = mean_absolute_percentage_error(test, pred_varma)

    performance["Model"].append("VARMA")
    performance["RMSE"].append(rmse_varma)
    performance["MaPe"].append(mape_varma)
    performance["Lag"].append(best_order)
    performance["Test"].append(test_obs)

    perf_varma = pd.DataFrame(performance)
    return perf_varma, results_varma, pred_varma

In [10]:
listt_varma = ["Close", "High", "Open", "Low"]

In [11]:
perf_varma, result_varma, pred_varma = varma_model(data1, listt_varma)

/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


['Close', 'High', 'Open', 'Low']


/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/statsmodels/base/model.py:607: Convergen

Best Order (p, q): (3, 1)


/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


In [12]:
print(perf_varma)

   Model  RMSE      MaPe     Lag  Test
0  VARMA  0.03  0.030552  (3, 1)    28


In [13]:
data1

,Open,High,Low,Close
0,0.517223,0.485749,0.529877,0.522210
1,0.500522,0.491400,0.530969,0.519394
2,0.498434,0.472236,0.528786,0.517972
3,0.506785,0.472236,0.532251,0.520225
4,0.507411,0.479533,0.527149,0.513788
...,...,...,...,...
2139,0.877349,0.819410,0.900409,0.880633
2140,0.856733,0.807862,0.885130,0.870440
2141,0.848643,0.799017,0.873670,0.865612
2142,0.864301,0.810565,0.892224,0.873927


In [15]:
perf_varma

,Model,RMSE,MaPe,Lag,Test
0,VARMA,0.03,0.030552,"(3, 1)",28
